# Deeper Networks for Image Classification


## MobileNetV2 Model for Image Classification

- Code by: Kaviraj Gosaye
- Student ID: 220575371

### 0. Imports

In [ ]:
# import libraries
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from torchinfo import summary
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
import seaborn as sns
import torchvision.ops.misc as misc
from datetime import datetime

### 1. Data Loading and Preprocessing

In [ ]:
# transform PIL image to tensor and normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load CIFAR10 dataset
train_set = torchvision.datasets.CIFAR10(root='../datasets/CIFAR10', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='../datasets/CIFAR10', train=False, download=True, transform=transform)
                                    
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=8)

In [ ]:
# extract one sample from the training set
dataiter = iter(train_loader)
images, labels = next(dataiter)

# plot the image
def imshow(img):
    # reverse normalization
    img = img / 2 + 0.5
    # convert tensor to numpy array
    npimg = img.numpy()
    # rearrange the dimensions to match matplotlib format
    # matplotlib:   H x W x C
    # torch:        C x H x W
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# show images
imshow(torchvision.utils.make_grid(images))

### 2. Model Building

In [ ]:
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            layers.append(misc.Conv2dNormActivation(inp, hidden_dim, kernel_size=1, norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU6))
        layers.extend([
            misc.Conv2dNormActivation(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim, norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU6),
            misc.Conv2dNormActivation(hidden_dim, oup, kernel_size=1, norm_layer=nn.BatchNorm2d, activation_layer=nn.Identity)
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, width_mult=1.0):
        super(MobileNetV2, self).__init__()
        self.cfgs = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        input_channel = 32
        layers = [misc.Conv2dNormActivation(3, input_channel, kernel_size=3, stride=2, padding=1, norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU6)]

        for t, c, n, s in self.cfgs:
            output_channel = int(c * width_mult)
            for i in range(n):
                stride = s if i == 0 else 1
                layers.append(InvertedResidual(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel

        self.features = nn.Sequential(*layers)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(input_channel, num_classes)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
# setting the device to cuda if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# creating instance of model and setting it to the device
mobilenetv2 = MobileNetV2(num_classes=10)

In [ ]:
# visualize the model
info = summary(mobilenetv2, (1, 3, 32, 32), col_names = ('input_size', 'output_size', 'num_params', 'kernel_size'))
print(info)

### 3. Model Training

In [ ]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenetv2.parameters(), lr=0.001)

# open log file in write mode
log_file = open("../logs/mobilenetv2_cifar10_log.txt", "w")

# training the model
start = time.time()

num_epochs = 50
losses = []
train_accs = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = mobilenetv2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # cumulative loss
        running_loss += loss.item()
        # printing the average loss every 100 mini-batches
        if i % 100 == 99:
            log_file.write(f"{timestamp} - [{epoch + 1}, {i + 1}] loss: {running_loss / 100}\n")
            print(f"{timestamp} - [{epoch + 1}, {i + 1}] loss: {running_loss / 100}")
            running_loss = 0.0

    correct = 0.0
    pred = outputs.argmax(dim=1, keepdim=True)
    # reshaping the labels to match the shape of the predictions
    # comparing the predictions to the labels using element-wise comparison
    # summing the correct predictions
    correct += pred.eq(labels.view_as(pred)).sum().item()
    train_acc = 100. * correct / len(outputs)
    train_accs.append(train_acc)
    losses.append(loss.item())
    
print(f"Finished Training after {time.time() - start} seconds")

# Close the log file
log_file.close()

In [ ]:
# save model after training
torch.save(mobilenetv2, "../Models/mobilenet_v2_cifar_10.pth")

### 4. Model Evaluation

In [ ]:
mobilenetv2.eval()
# Disable gradient calculation
with torch.no_grad():
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    test_loss = 0

    # Using test set
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        
        # Forward pass
        outputs = mobilenetv2(images)
        
        # Calculate the test loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Append the predicted and true labels
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    # Calculate the accuracy
    accuracy = 100 * correct / total
    test_loss /= len(test_loader.dataset)

# Print the accuracy and test loss
print(f"Accuracy on the test data: {accuracy}%")
print(f"Test Loss: {test_loss}")

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix - MobileNetV2 on CIFAR-10")
plt.show()

In [ ]:
# Plot the loss
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss of MobileNetV2 on CIFAR-10')
plt.show()

In [ ]:
# Plot the accuracy
plt.plot(train_accs)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy of MobileNetV2 on CIFAR-10')
plt.show()

In [ ]:
# Convert lists to tensors
pred_labels_tensor = torch.tensor(predicted_labels)
true_labels_tensor = torch.tensor(true_labels)

# Calculate precision and recall
precision = precision_score(true_labels_tensor, pred_labels_tensor, average='weighted')
recall = recall_score(true_labels_tensor, pred_labels_tensor, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

### 5. Model Testing

In [ ]:
# loading some images from the test set
testloader = torch.utils.data.DataLoader(test_set, batch_size=30,shuffle=False)

dataiter = iter(testloader)
images, true_labels = next(dataiter)   

In [ ]:
# classes of CIFAR-10
classes = ('Plane', 'Car', 'Bird', 'Cat','Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

In [ ]:
# plotter function to plot the images
def plotter(images, true_labels, predicted_labels):
    correct = 0
    num_images = len(images)
    num_rows = (num_images) // 5
    fig, axs = plt.subplots(num_rows, 5, figsize=(15, 20))
    # setting the title
    fig.suptitle('True Label - [Predicted Labels]', fontsize=20)

    for ind in range(num_images):
        # plot image
        img = images[ind].permute(1, 2, 0) / 2 + 0.5
        ax = axs[ind // 5, ind % 5]
        ax.imshow(img)

        # left - true label, right - predicted label
        if classes[true_labels[ind]] == classes[predicted_labels[ind]]:
            color = 'green'
            title = classes[true_labels[ind]] + ' - [' + classes[predicted_labels[ind]] + ']'
            ax.set_title(title, color=color)
            correct += 1
        else:
            color = 'red'
            title = classes[true_labels[ind]] + ' - [' + classes[predicted_labels[ind]] + ']'
            ax.set_title(title, color=color)        

    plt.tight_layout(pad=2.0)
    plt.show()
    print('Number of correct predictions: ', correct)
    print('Number of wrong predictions: ', num_images - correct)

In [ ]:
# use model to predict images label
mobilenetv2.eval()
mobilenetv2.to('cpu')
outputs = mobilenetv2(images)
_, predicted_labels = torch.max(outputs, 1)

In [ ]:
# plot the images and predictions
plotter(images, true_labels, predicted_labels)